# NIKA2 Run preparation

This notebook shows how to compute the primary calibrator fluxes for a NIKA2 run. These are the planets
* Mars
* Uranus
* Neptune

For each calibrator, we will write a text file containing the values

In [1]:
import photometry as pt
from astropy.time import Time, TimeDelta
from astropy import units as u
from astropy.table import Table
import numpy as np
import os
import re

In [2]:
# get the planet objects
uranus = pt.GiantPlanet('Uranus', model_version='esa4')
neptune = pt.GiantPlanet('Neptune', model_version='esa5')
mars = pt.Mars()
# and the NIKA2 passbands
nika2mm = pt.Passband(file='2mm.NIKA2.pb')
nika1mmH = pt.Passband(file='1mmH.NIKA2.pb')
nika1mmV = pt.Passband(file='1mmV.NIKA2.pb')

In [3]:
# Table with the run informations

In [4]:
workdir = '/Users/haussel/work/nika2/commissioning/run_calibrations/'
runinfo = os.path.join(workdir, 'runinfo.txt')

In [5]:
# Time step for the computations, we use 1 day.
deltat = TimeDelta(1, format='jd')

In [6]:
tabrun = Table.read(runinfo, comment='#', format='ascii')

In [7]:
tabrun

RunID,StartDate,EndDate
str6,str10,str10
run22,2017-02-20,2017-02-26
run23,2017-04-17,2017-04-26
run24,2017-06-07,2017-06-14
run27,2018-01-12,2018-01-21
run28,2018-02-12,2018-02-21
run29,2018-03-02,2018-03-21
run30,2018-05-21,2018-05-30
run31,2018-06-11,2018-06-20
run32,2018-09-04,2018-09-07


In [8]:
tabrun['StartDate'] = Time(tabrun['StartDate'])
tabrun['EndDate'] = Time(tabrun['EndDate'])
tabrun['Duration'] = tabrun['EndDate'] - tabrun['StartDate']

In [9]:
alldates = []
for row in tabrun:
    dates = row['StartDate'] + deltat * np.arange(0, 
                                                  row['Duration'].value + 1,
                                                  1)
    alldates = np.append(alldates, dates.value)
alldates = Time(alldates, format='iso')

In [11]:
print(alldates)

['2017-02-20 00:00:00.000' '2017-02-21 00:00:00.000'
 '2017-02-22 00:00:00.000' '2017-02-23 00:00:00.000'
 '2017-02-24 00:00:00.000' '2017-02-25 00:00:00.000'
 '2017-02-26 00:00:00.000' '2017-04-17 00:00:00.000'
 '2017-04-18 00:00:00.000' '2017-04-19 00:00:00.000'
 '2017-04-20 00:00:00.000' '2017-04-21 00:00:00.000'
 '2017-04-22 00:00:00.000' '2017-04-23 00:00:00.000'
 '2017-04-24 00:00:00.000' '2017-04-25 00:00:00.000'
 '2017-04-26 00:00:00.000' '2017-06-07 00:00:00.000'
 '2017-06-08 00:00:00.000' '2017-06-09 00:00:00.000'
 '2017-06-10 00:00:00.000' '2017-06-11 00:00:00.000'
 '2017-06-12 00:00:00.000' '2017-06-13 00:00:00.000'
 '2017-06-14 00:00:00.000' '2018-01-12 00:00:00.000'
 '2018-01-13 00:00:00.000' '2018-01-14 00:00:00.000'
 '2018-01-15 00:00:00.000' '2018-01-16 00:00:00.000'
 '2018-01-17 00:00:00.000' '2018-01-18 00:00:00.000'
 '2018-01-19 00:00:00.000' '2018-01-20 00:00:00.000'
 '2018-01-21 00:00:00.000' '2018-02-12 00:00:00.000'
 '2018-02-13 00:00:00.000' '2018-02-14 00:00:0

In [12]:
# compute the Neptune and Uranus Fluxes at the reference frequencies of each band
neptune.set_dates(alldates)
uranus.set_dates(alldates)
spec_uranus = uranus.spectral_irradiance()
spec_neptune = neptune.spectral_irradiance()

fluxes_1mm_uranus= spec_uranus.fnu_nu(nika1mmH.xref(u.GHz)).to(u.Jy)
fluxes_1mm_neptune = spec_neptune.fnu_nu(nika1mmH.xref(u.GHz)).to(u.Jy)
fluxes_2mm_uranus = spec_uranus.fnu_nu(nika2mm.xref(u.GHz)).to(u.Jy)
fluxes_2mm_neptune = spec_neptune.fnu_nu(nika2mm.xref(u.GHz)).to(u.Jy)

In [13]:
# compute Mars fluxes at the reference frequencies for each run
mars.set_beam('Iram30m')
mars.set_instrument('NIKA2')

<Quantity [ 150.,  260., 1600., 3200.] GHz>

In [14]:
Fnu150GHzMars = np.zeros(len(alldates))
Fnu260GHzMars = np.zeros(len(alldates))
for i, date in enumerate(alldates):
    Fnu150GHzMars[i] = mars.fnu(150 * u.GHz, date).value
    Fnu260GHzMars[i] = mars.fnu(260 * u.GHz, date).value

In [15]:
alldates

<Time object: scale='utc' format='iso' value=['2017-02-20 00:00:00.000' '2017-02-21 00:00:00.000'
 '2017-02-22 00:00:00.000' '2017-02-23 00:00:00.000'
 '2017-02-24 00:00:00.000' '2017-02-25 00:00:00.000'
 '2017-02-26 00:00:00.000' '2017-04-17 00:00:00.000'
 '2017-04-18 00:00:00.000' '2017-04-19 00:00:00.000'
 '2017-04-20 00:00:00.000' '2017-04-21 00:00:00.000'
 '2017-04-22 00:00:00.000' '2017-04-23 00:00:00.000'
 '2017-04-24 00:00:00.000' '2017-04-25 00:00:00.000'
 '2017-04-26 00:00:00.000' '2017-06-07 00:00:00.000'
 '2017-06-08 00:00:00.000' '2017-06-09 00:00:00.000'
 '2017-06-10 00:00:00.000' '2017-06-11 00:00:00.000'
 '2017-06-12 00:00:00.000' '2017-06-13 00:00:00.000'
 '2017-06-14 00:00:00.000' '2018-01-12 00:00:00.000'
 '2018-01-13 00:00:00.000' '2018-01-14 00:00:00.000'
 '2018-01-15 00:00:00.000' '2018-01-16 00:00:00.000'
 '2018-01-17 00:00:00.000' '2018-01-18 00:00:00.000'
 '2018-01-19 00:00:00.000' '2018-01-20 00:00:00.000'
 '2018-01-21 00:00:00.000' '2018-02-12 00:00:00.000'
 

In [16]:
ymd = re.compile('(\d\d\d\d)-(\d\d)-(\d\d)')

In [17]:
alldays = []
for date in alldates.value:
    m = ymd.match(date)
    if m:
        alldays.append('{}{}{}'.format(m.group(1), m.group(2), 
                                       m.group(3)))
    else:
        raise ValueError('Cannot parse date {}'.format(date))

In [18]:
# write the output for uranus
out_uranus = os.path.join(workdir, 'uranus.txt')
with open(out_uranus, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz\n")
    for i, day in enumerate(alldays):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, 
                                                 fluxes_2mm_uranus[i].value, 
                                                 fluxes_1mm_uranus[i].value))
fout.close()

In [19]:
# write the output for neptune
out_neptune = os.path.join(workdir, 'neptune.txt')
with open(out_neptune, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz\n")
    for i, day in enumerate(alldays):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, 
                                                 fluxes_2mm_neptune[i].value, 
                                                 fluxes_1mm_neptune[i].value))
fout.close()

In [20]:
# write the output for mars
out_mars = os.path.join(workdir, 'mars.txt')
with open(out_mars, 'w') as fout:
    fout.write("# Date Fnu_150GHz   Fnu_260GHz\n")
    for i, day in enumerate(alldays):
        fout.write("{} {:5.3f} {:5.3f}\n".format(day, 
                                                 Fnu150GHzMars[i], 
                                                 Fnu260GHzMars[i]))
fout.close()

In [21]:
Fnu150GHzMars

array([  59.494,   59.172,   58.799,   58.366,   57.907,   57.456,
         57.027,   41.94 ,   41.593,   41.23 ,   40.855,   40.476,
         40.143,   39.83 ,   39.559,   39.331,   39.127,   33.578,
         33.654,   33.732,   33.795,   33.826,   33.806,   33.745,
         33.64 ,   64.801,   65.59 ,   66.469,   67.417,   68.407,
         69.432,   70.467,   71.477,   72.451,   73.368,   91.897,
         92.642,   93.454,   94.358,   95.416,   96.562,   97.865,
         99.319,  100.93 ,  102.679,  119.064,  120.546,  122.006,
        123.511,  125.098,  126.806,  128.662,  130.666,  132.784,
        134.92 ,  137.19 ,  139.438,  141.609,  143.684,  145.626,
        147.45 ,  149.215,  150.968,  152.72 ,  154.492,  495.025,
        506.079,  517.634,  529.974,  542.321,  555.393,  568.579,
        581.585,  594.272,  606.782,  771.306,  789.861,  809.597,
        830.576,  852.571,  875.355,  898.735,  922.241,  945.56 ,
        968.681, 1082.013, 1064.143, 1045.842, 1027.078,  850.